# **7. Reentrenamiento** **del** **modelo** **YOLO** **para** **detección** **de** **objetos**

En este módulo se lleva a cabo el reentrenamiento (fine-tuning) de un modelo YOLO para la detección de ingredientes. Partimos del checkpoint *best4.pt*, procedente del proyecto *lannguyen0910/food-recognition*, y lo adaptamos a los objetivos específicos de este trabajo.

El modelo original fue entrenado con un dataset mayoritariamente de recetas asiáticas, por lo que su rendimiento no resulta satisfactorio con ingredientes comunes en la cocina española (por ejemplo, ajo y cebolla). En consecuencia, no es adecuado para su uso directo en este proyecto y requiere un proceso de ajuste.

Dado el alcance limitado del proyecto, el fine tuning se hace únicamente con tres categororías: *'egg'*, *'potato'* y *'cebolla'*. La categoría cebolla no esá incluida en el modelo de partida, incorporandose a éste con el reentrenamiento.

In [ ]:
!pip install roboflow
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 102.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.1 MB/s eta 0:00:00


Para el fine-tuning del modelo empleamos un dataset propio, anotado con cajas bounding boxes. Para hacer esta anotación se ha utilizado la plataforma *Roboflow*.

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="Io6wF1fvJQcDglq2hhyB")
project = rf.workspace("jorge-xtokd").project("anotate_food-fdzy3")
version = project.version(7)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to anotate_food-7 in yolov8:: 100%|██████████| 439/439 [00:00<00:00, 6999.67it/s]


Las clases anotadas en el datset de reentrenamiento y sus respectivos índices son los siguientes.

In [ ]:
import yaml

# Ruta al data.yaml de tu dataset
data_yaml_path = "/content/anotate_food-7/data.yaml"  # Ajusta la ruta

# Cargar y leer
with open(data_yaml_path, "r") as f:
    data = yaml.safe_load(f)

names = data.get("names", [])
print("Clases anotadas en el dataset:\n")

# Si es lista
if isinstance(names, list):
    for idx, name in enumerate(names):
        print(f"{idx}: {name}")

# Si es dict
elif isinstance(names, dict):
    for idx, name in names.items():
        print(f"{idx}: {name}")

Clases anotadas en el dataset:

0: egg
1: onion
2: potato


El siguiente paso es integrar las clases utilizadas para reentrenar el modelo en el modelo base. Para ello, se deben de remapear los IDs de clases para que sean coherentes con los índices existentes del modelo base (índices 43 para 'egg', 61 para 'potato', 90 para 'onion').

In [ ]:
from pathlib import Path
import yaml

# === Ajusta rutas y mapeo aquí ===
dataset_path = Path("/content/anotate_food-7")  # durante el proceso iterativo de reentrenamiento se han utilizado 7 datasets diferentes, actualizados en cada iteración para afinar el modelo.
id_map = {0: 43, 2: 61, 1: 90}  # antiguo_id: nuevo_id

# 1) Reasignar índices en etiquetas
for split in ["train", "valid", "val", "test"]:
    labels_dir = dataset_path / split / "labels"
    if not labels_dir.exists():
        continue
    for file in labels_dir.glob("*.txt"):
        with open(file, "r") as f:
            lines = f.readlines()
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if not parts:
                continue
            cid = int(parts[0])
            if cid in id_map:
                parts[0] = str(id_map[cid])  # reasigna el índice
            new_lines.append(" ".join(parts) + "\n")
        with open(file, "w") as f:
            f.writelines(new_lines)

print("✔ Etiquetas actualizadas con nuevo índice.")

# 2) Reasignar índices en data.yaml
data_yaml_path = dataset_path / "data.yaml"
with open(data_yaml_path, "r") as f:
    data = yaml.safe_load(f)

# Si 'names' es lista → convertir a diccionario para poder reasignar índices arbitrarios
names = data.get("names", [])
if isinstance(names, list):
    names = {i: names[i] for i in range(len(names))}

# Aplicar el mapeo
for old, new in id_map.items():
    if old in names:
        names[new] = names.pop(old)

data["names"] = names  # ahora es un diccionario con los nuevos índices

with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data, f, sort_keys=True)

print("✔ data.yaml actualizado con nuevo índice.")

✔ Etiquetas actualizadas con nuevo índice.
✔ data.yaml actualizado con nuevo índice.


Se contabilizan las anotaciones de cada clase presentes en el dataset de entrenamiento.

In [ ]:
from collections import Counter

# Ruta base de tu dataset
dataset_path = Path("/content/anotate_food-7")  # ajusta a tu ruta

# Leer nombres de clases del data.yaml
data_yaml_path = dataset_path / "data.yaml"
with open(data_yaml_path, "r") as f:
    data = yaml.safe_load(f)

names = data.get("names", {})
# Si names está como lista, convertir a dict para tener ids explícitos
if isinstance(names, list):
    names = {i: names[i] for i in range(len(names))}

# Contador de anotaciones por clase
counts = Counter()

# Recorrer etiquetas de todos los splits
for split in ["train", "valid", "val", "test"]:
    labels_dir = dataset_path / split / "labels"
    if not labels_dir.exists():
        continue
    for file in labels_dir.glob("*.txt"):
        with open(file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if parts:
                    cid = int(parts[0])
                    counts[cid] += 1

# Mostrar resumen ordenado por ID
print("Resumen de anotaciones por clase:\n")
for cid in sorted(names.keys()):
    name = names[cid]
    n = counts[cid]
    print(f"ID {cid:>3}: {name:<20} -> {n} anotaciones")

Resumen de anotaciones por clase:

ID  43: egg                  -> 258 anotaciones
ID  61: potato               -> 710 anotaciones
ID  90: onion                -> 177 anotaciones


Carga de los pesos del modelo *best4.pt* que utilizaremos como base de finetuning para obtener nuestro modelo.

In [ ]:
from google.colab import files

# Esto abrirá un diálogo para elegir tu archivo .pt desde tu PC
uploaded = files.upload()

Saving best4.pt to best4.pt


Dado que el modelo base no incluye la clase 'onion' que queremos entrenar, es necesario añadirla al dataset. Para ello, se utiliza este código, que actualiza el archivo *data.yaml* con el nuevo listado de clases.

Se aprovecha en el mismo código para cargar carga los pesos del modelo base (*best4.pt*) y verifica que las clases originales coincidan con lo esperado (90 clases en total).

In [ ]:
from ultralytics import YOLO


weights_path = "/content/best4.pt"
dataset_root = Path("/content/anotate_food-7")
val_dir = "valid" if (dataset_root/"valid").exists() else "val"

# Cargar modelo y extraer nombres (0..89)
model = YOLO(weights_path)
names_old = model.names
names = [names_old[i] for i in range(max(names_old.keys())+1)]
assert len(names) == 90, f"Esperaba 90 clases, hay {len(names)}"

# Añadir NUEVA clase → ID 90
NEW_CLASS_NAME = "onion"
names.append(NEW_CLASS_NAME)       # ahora 91 clases (0..90)

# Escribir data.yaml
data_yaml = {
    "path": str(dataset_root),
    "train": "train",
    "val": val_dir,
    "names": names,
    "nc": len(names)   # 91
}
data_yaml_path = dataset_root/"data.yaml"
with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_yaml, f, sort_keys=False)

print("data.yaml listo con 91 clases (0..90).")
print(f"Clase 43: {names[43]}  |  Clase 61: {names[61]}  |  Clase 90: {names[90]}")


data.yaml listo con 91 clases (0..90).
Clase 43: Egg  |  Clase 61: Potato  |  Clase 90: onion


Reentrenamiento del modelo, con congelación de las primeras capas de la red para dar mayor estabilidad al entrenamiento y prevenir un  sobreajuste temprano.

In [ ]:
results = model.train(
    data=str(data_yaml_path),
    epochs=10,
    lr0=8e-4,
    imgsz=640,
    batch=8,
    freeze=10,     # congela parte del backbone para estabilidad
    patience=10,   # early stopping
    mosaic=1.0,
    fliplr=0.5,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4
)

Ultralytics 8.3.188 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/anotate_food-7/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0008, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/best4.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0,

In [ ]:
import glob, shutil, os

# Buscar todas las carpetas de entrenamiento en runs/detect/
runs = sorted(glob.glob("runs/detect/train*"))
if not runs:
    raise FileNotFoundError("No se encontraron entrenamientos en runs/detect/")

# Tomar el último entrenamiento
last_run = runs[-1]
best_model = os.path.join(last_run, "weights", "best.pt")

# Copiar a un nombre más claro
shutil.copy(best_model, "/content/best_new.pt")
print("Modelo guardado como /content/best_new.pt")

Modelo guardado como /content/best_new.pt


In [ ]:
from google.colab import files
files.download("/content/best_new.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>